In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
os.getcwd()

'C:\\Users\\DarioAlejandroLatorr\\OneDrive\\claro_20_abril_2021\\ENVIO_DARIO\\TRONCALES_HEAVYUSERS_DATAGEO'

In [3]:
pd.read_csv('Datos_heavy_users/USERS_CLASIFICADOS_COMPLETO.csv',nrows=1, sep='|').columns

Index(['CUENTA', 'CUENTA_MATRIZ', 'ESTRATO', 'NODO', 'CIUDAD', 'NOMBRE_NODO',
       'COMUNIDAD', 'DES_DIR_ESTAN', 'DES_DIR_NUEVA', 'LATITUD', 'LONGITUD',
       'DES_AGREGACION', 'DES_DEPARTAMENTO', 'COD_DANE', 'DES_CIUDAD',
       'DES_SEGMENTO', 'flbb_down_traffic_bytes', 'flbb_up_traffic_bytes',
       'traf_total', 'TIPO_USUARIO', 'TIPO_USUARIO_SIMP', 'TIPO_USUARIO_UP',
       'TIPO_USUARIO_UP_SIMP'],
      dtype='object')

In [4]:
def preparar_heavy_users(dw_up_total, percentil=99, plot_scatter=True):
    """
    dw_up_total debe ser alguno de los tres valores: 'up', 'down', 'total'
    
    percentil corresponde al percentil de la distribución de tráficos a partir de la que 
    se consideran usuarios heavy.
    
    Los datos de los usuarios se importan de la carpeta Datos_heavy_users
    y el archivo USERS_CLASIFICADOS_COMPLETO.csv 
    """
    # columnas para armar el archivo
    cols_list = ['CUENTA', 'CUENTA_MATRIZ', 'ESTRATO', 'NODO', 'CIUDAD', 'NOMBRE_NODO','LATITUD', 'LONGITUD']
    dict_traf = {'down':'flbb_down_traffic_bytes', 'up':'flbb_up_traffic_bytes','total':'traf_total'}
    col_traffic = dict_traf[dw_up_total]
    cols_list += [col_traffic] 
    
    ruta = 'Datos_heavy_users/USERS_CLASIFICADOS_COMPLETO.csv'
    df_users = pd.read_csv(ruta, usecols=cols_list,sep='|')
    l_total = len(df_users)
    
    # agrupando por nodo, contando nodos y sumando traficos sobre el dataframe entero 
    agg_nodos = df_users.groupby(['NODO','NOMBRE_NODO']).agg({dict_traf[dw_up_total]:['sum','count']}).reset_index()
    agg_nodos.columns = agg_nodos.columns.droplevel()
    agg_nodos.columns = ['NODO','NOMBRE_NODO',f'{dw_up_total}_traffic_sum_nodo',f'{dw_up_total}_cantidad_users_nodo']
    
    # filtrando y dejando solo un porcentaje de los usuarios con más tráfico
    per_traf = np.percentile(df_users[col_traffic],percentil)
    df_users = df_users[(df_users[col_traffic]>per_traf)].copy()
    
    # uniendo la información del agrupamiento por nodo
    df_users = df_users.merge(agg_nodos, on=['NODO','NOMBRE_NODO'], how='left')
    
    print(f'Se seleccionan usuarios con tráficos {dw_up_total} mayores que {round(per_traf,2)}, {len(df_users)} usuarios de {l_total}.')
    
    # columnas de proporción  y número de usuarios preparadas para los gráficos
    y_plot = f'{dw_up_total}_log_numero_usuarios'
    size_plot = f'{dw_up_total}_participacion_trafico'
    df_users[y_plot] = np.log(df_users[f'{dw_up_total}_cantidad_users_nodo'])
    df_users[size_plot] = ((df_users[dict_traf[dw_up_total]])/(df_users[f'{dw_up_total}_traffic_sum_nodo']))
    
    
    df_users.to_csv(f'Datos_heavy_users/heavy_user_{dw_up_total}_per_{percentil}_excel.csv', sep=';', decimal=',', index=False)
    
    if plot_scatter:
        plt.figure()
        sns.scatterplot(data=df_users, x=col_traffic, y=y_plot, size=size_plot)
        plt.show()

In [5]:
preparar_heavy_users('down', percentil=99, plot_scatter=False)
preparar_heavy_users('up', percentil=99, plot_scatter=False)
preparar_heavy_users('total', percentil=99, plot_scatter=False)

Se seleccionan usuarios con tráficos down mayores que 45.9, 26315 usuarios de 2631427.
Se seleccionan usuarios con tráficos up mayores que 4.34, 26315 usuarios de 2631427.
Se seleccionan usuarios con tráficos total mayores que 49.61, 26315 usuarios de 2631427.
